In [101]:
import firebase_admin
from firebase_admin import credentials, firestore
from collections import Counter
import pandas as pd
import pydeck as pdk
from geopy.geocoders import Nominatim
import time
import webbrowser
import tempfile

def fetch_review_indexes(email):
    """이메일에 해당하는 Firestore의 인덱스를 조회."""
    # Firestore 클라이언트 생성
    db = firestore.client()
    
    # 'review' 컬렉션 참조
    collection_ref = db.collection('review')
    
    # 이메일로 필터링된 쿼리 생성
    query = collection_ref.where('email', '==', email).stream()
    
    # 인덱스 필드를 가져오기
    indexes = [doc.to_dict().get('index') for doc in query if 'index' in doc.to_dict()]
    
    return indexes

def initialize_firebase():
    """Firebase Admin SDK 초기화."""
    if not len(firebase_admin._apps):
        # Firebase 인증서 파일 경로
        cred = credentials.Certificate('C:/pododoc/key/kosmo-96bbe-60906db745e9.json')
        firebase_admin.initialize_app(cred)

def load_country_data(file_path):
    """CSV 파일에서 국가 데이터를 로드."""
    df = pd.read_csv(file_path)
    return df.set_index('index')['wine_country'].to_dict()

def get_location_from_country(country_name):
    """국가 이름으로부터 위도와 경도를 가져옵니다."""
    geolocator = Nominatim(user_agent="country_locator")
    location = geolocator.geocode(country_name)
    if location:
        return location.latitude, location.longitude
    return None, None

def create_deck_map(indexes, country_data):
    """pydeck을 사용하여 3D 구 시각화 생성"""
    # 국가별 빈도 계산
    counter = Counter(indexes)
    
    # 빈도의 범위를 설정하여 구의 크기를 조정합니다.
    min_size = 10000
    max_size = 50000
    max_count = max(counter.values())
    
    # pydeck Layer 데이터 준비
    data = []
    for index, count in counter.items():
        country = country_data.get(index, 'Unknown')
        lat, lon = get_location_from_country(country)
        if lat and lon:
            # 원의 크기를 빈도에 비례하도록 설정
            size = min_size + (count / max_count) * (max_size - min_size)
            data.append({
                'latitude': lat,
                'longitude': lon,
                'size': size,
                'country': country,
                'count': count
            })
            time.sleep(1)  # 지오코딩 요청의 빈도 제한을 피하기 위한 지연 시간
    
    # pydeck Deck 설정
    deck = pdk.Deck(
        layers=[
            pdk.Layer(
                'ScatterplotLayer',
                data=data,
                get_position='[longitude, latitude]',
                get_radius='size',
                radius_scale=1,
                radius_min_pixels=10,
                radius_max_pixels=100,
                get_fill_color='[0, 0, 255, 200]',
                pickable=True,
                auto_highlight=True,
            ),
        ],
        initial_view_state=pdk.ViewState(
            latitude=0,
            longitude=0,
            zoom=2,
            pitch=40.5,
            bearing=0
        ),
        tooltip={
            'text': '{country}: {count}'
        }
    )

    # Deck 렌더링
    with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as f:
        deck.to_html(f.name)
        webbrowser.open(f.name)  # 기본 웹 브라우저에서 열기

# Firebase 초기화
initialize_firebase()

# 이메일 읽기
with open('received_email.txt', 'r') as file:
    email = file.read().strip()

# 인덱스 조회
indexes = fetch_review_indexes(email)

# 국가 데이터 로드
country_data = load_country_data('data/Combined_Wine_Data.csv')

# 3D 지도 생성
create_deck_map(indexes, country_data)


c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
